<a href="https://colab.research.google.com/github/marquezjp/SQLToPandas/blob/main/Oracle/Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exportar uma Query do Oracle para um DataFrame

## Preparar Ambiente

Para instalar o [cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)

[Download](https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html) do [Oracle Instant Client Basic Light](https://download.oracle.com/otn_software/nt/instantclient/19900/instantclient-basiclite-windows.x64-19.9.0.0.0dbru.zip)

```
!pip install cx_Oracle --upgrade
```

```
cx_Oracle.init_oracle_client(lib_dir=r"P:\Oracle\InstantClient")
```

Para instalar o XlsxWriter

```
!pip install -U XlsxWriter
```

Para instalar o Beautiful Soup

```
!pip install --upgrade beautifulsoup4
```

In [35]:
# Carrega a Biblioteca Basicas
from datetime import date
from io import BytesIO
import csv

In [2]:
# Carrega a Biblioteca Pandas
import pandas as pd

In [3]:
# Carrega a Biblioteca Excel
import xlsxwriter

In [4]:
# Carrega a Biblioteca Oracle, SQLite e SQL Alchemy
#import sqlite3
import cx_Oracle
from sqlalchemy.engine import create_engine

In [5]:
today = date.today().strftime("%d/%m/%Y")
print("Hoje é", today)

Hoje é 05/11/2021


## Conexão via SQL Alchemy

In [6]:
# Definir o Diretorio do Oracle Client Libraries
cx_Oracle.init_oracle_client(lib_dir=r"P:\Oracle\InstantClient")

# Cria a Conexão do Oracle
oracleConfig = {
    'dialect': 'oracle',
    'sql_driver': 'cx_oracle',
    'dsn_tns': {
        'host': '192.168.10.247',
        'service': 'sigrhprod.financas.pref',
        'port': 1521},
    'user': 'sigrhprd',
    'password': 'prdM4c3i0'
}

dialeto = oracleConfig.get('dialect', 'oracle')
sql_driver = oracleConfig.get('sql_driver', 'cx_oracle')

dsn_tnsConfig = oracleConfig.get('dsn_tns', {'host': '', 'service': '', 'port': 1521})
dsn_tns = cx_Oracle.makedsn(dsn_tnsConfig.get('host', '0.0.0.0'),
                            dsn_tnsConfig.get('port', '1521'),
                            service_name=dsn_tnsConfig.get('service', 'orcl'))

user=oracleConfig.get('user', '')
password=oracleConfig.get('password', '')

conexaoOracle = create_engine(f'{dialeto}+{sql_driver}://{user}:{password}@{dsn_tns}')
conexaoOracle

Engine(oracle+cx_oracle://sigrhprd:***@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=192.168.10.247)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=sigrhprod.financas.pref))))

## Consulta SQL

### ContraCheque Simplificado

In [7]:
# ContraCheque Simplificado
SQL = '''
select 
 o.sgorgao as Orgao,
 f.nuanoreferencia || lpad(f.numesreferencia,2,0) as AnoMes,
 lpad(v.numatricula, 7, 0)||'-'||v.nudvmatricula as Matricula,
 lpad(p.nucpf,11,0) as CPF,
 p.nmpessoa as Nome,

 nvl2(capa.cdcargocomissionado, 'COMISSIONADO', itemnv1.deitemcarreira) as Carreira,
 nvl(d.decargocomissionado, item.deitemcarreira) as Cargo,

 lpad(rub.cdtiporubrica,2,0) || '-' || lpad(rub.nurubrica,4,'0') || '-' || lpad(pag.nusufixorubrica,2,'0') as Rubrica,
 rub.derubricaagrupamento as DeRubrica,
 pag.vlpagamento as Valor

from epaghistoricorubricavinculo pag
inner join epagfolhapagamento f on f.cdfolhapagamento = pag.cdfolhapagamento and f.flcalculodefinitivo = 'S'
                               and f.cdtipofolhapagamento = 2 and f.cdtipocalculo = 1
inner join epagcapahistrubricavinculo capa on capa.cdfolhapagamento = pag.cdfolhapagamento and capa.cdvinculo = pag.cdvinculo
inner join vcadorgao o on o.cdorgao = f.cdorgao

inner join ecadvinculo v on v.cdvinculo = pag.cdvinculo 
inner join ecadpessoa p on p.cdpessoa = v.cdpessoa

left join ecadevolucaocargocomissionado d on d.cdcargocomissionado = capa.cdcargocomissionado

left join ecadestruturacarreira estr on estr.cdestruturacarreira = capa.cdestruturacarreira
left join ecaditemcarreira item on item.cditemcarreira = estr.cditemcarreira
left join ecadestruturacarreira estrnv1 on estrnv1.cdestruturacarreira = estr.cdestruturacarreirapai
left join ecaditemcarreira itemnv1 on itemnv1.cditemcarreira = estrnv1.cditemcarreira

inner join vpagrubricaagrupamento rub on rub.cdrubricaagrupamento = pag.cdrubricaagrupamento

where f.nuanoreferencia = 2021 and f.numesreferencia = 07
  and rub.cdtiporubrica != 9
  and o.sgorgao = 'SEMGE'
  
order by o.sgorgao, v.numatricula, rub.cdtiporubrica, rub.nurubrica, pag.nusufixorubrica
'''

## Executar a Consulta

### Executar a Consulta na Base Oracle

In [8]:
%%time
base = pd.read_sql_query(SQL, conexaoOracle)
base.head()

Wall time: 4.57 s


,orgao,anomes,matricula,cpf,nome,carreira,cargo,rubrica,derubrica,valor
0,SEMGE,202107,0010153-2,56390068468,MARIA LUCIA DA SILVA SOUZA,ADMINISTRACAO GERAL - NIVEL ELEMENTAR 30H,AUXILIAR - SERVICOS GERAIS,01-0101-01,VENCIMENTO FIXO,1344.58
1,SEMGE,202107,0010153-2,56390068468,MARIA LUCIA DA SILVA SOUZA,ADMINISTRACAO GERAL - NIVEL ELEMENTAR 30H,AUXILIAR - SERVICOS GERAIS,01-0121-01,UNID AVAL DESEMP CENT/PESSOAL,900.00
2,SEMGE,202107,0010153-2,56390068468,MARIA LUCIA DA SILVA SOUZA,ADMINISTRACAO GERAL - NIVEL ELEMENTAR 30H,AUXILIAR - SERVICOS GERAIS,01-0190-01,ANUENIO,443.71
3,SEMGE,202107,0010153-2,56390068468,MARIA LUCIA DA SILVA SOUZA,ADMINISTRACAO GERAL - NIVEL ELEMENTAR 30H,AUXILIAR - SERVICOS GERAIS,05-0339-01,C.D.C.B.BRASIL,411.96
4,SEMGE,202107,0010153-2,56390068468,MARIA LUCIA DA SILVA SOUZA,ADMINISTRACAO GERAL - NIVEL ELEMENTAR 30H,AUXILIAR - SERVICOS GERAIS,05-0399-01,BRADESCO FINANCIAMENTOS,33.17


### Selecionar Campos da Consulta

In [9]:
campos = [
    ('orgao', 9, 'Codigo'), 
    ('anomes', 6, 'Codigo'),
    ('matricula', 9, 'Codigo'),
    ('cpf', 11, 'Codigo'),
    ('nome', 50, 'Descricao'),
    ('carreira', 30, 'Descricao'),
    ('cargo', 50, 'Descricao'),
    ('rubrica', 10, 'Codigo'),
    ('derubrica', 30, 'Descricao'),
    ('valor', 9, 'Valor')
]

In [10]:
camposSelecionados = [ campo[0] for campo in campos]
#print(base[camposSelecionados].iloc[0].values)
print(base[camposSelecionados].values[0])

['SEMGE' '202107' '0010153-2' '56390068468' 'MARIA LUCIA DA SILVA SOUZA'
 'ADMINISTRACAO GERAL - NIVEL ELEMENTAR 30H' 'AUXILIAR - SERVICOS GERAIS'
 '01-0101-01' 'VENCIMENTO FIXO' 1344.58]


## Salva os Dados Selecionados no Excel

**Table Style Medium**

**Cor**

* 1 e 15 - Branco
* 8 e 22 - Cinza Claro
* 2, 9, 16 e 23 - Azul
* 3, 10, 17 e 24 - Vermelho
* 4, 11, 18 e 25 - Verde Oliva
* 5, 12, 19 e 26 - Roxo
* 6, 13, 20 e 27 - Verde Agua
* 7, 14, 21 e 28 - Laranja

**Linhas Brancas (1 a 7, e 15 a 21)**

**Sem Cabeçalho (22 a 28)**


In [45]:
%%time
dataReferencia = date.today().strftime('%Y%m%d')
nomePlanilha = 'CONTRACHEQUE'
nomeArquivo = f'SIGRH-MCZ - {nomePlanilha} - {dataReferencia}.xlsx'

planilha = pd.ExcelWriter(nomeArquivo, engine='xlsxwriter', datetime_format='dd/mm/yy', date_format='dd/mm/yy')
base.to_excel(planilha, sheet_name=nomePlanilha, startrow=0, index=False)

# Formata Tabela
pagina = planilha.sheets[nomePlanilha]
linhas, colunas = base.shape
column_settings = [{'header': column.upper()} for column in base.columns.values.tolist()]
tabelaOpcoes = {
    'name': nomePlanilha,
    'style': 'Table Style Medium 15',
    'autofilter': True,
    'columns': column_settings
}
pagina.add_table(0, 0, linhas, colunas - 1, tabelaOpcoes)

vl_format = planilha.book.add_format({'align': 'right', 'num_format': '#,##0.00'})
de_format = planilha.book.add_format({'align': 'left'})
cd_format = planilha.book.add_format({'align': 'center'})

for campo, parm in enumerate(campos):
    tamanho = parm[1]
    if parm[2] == 'Descricao':
        formato = de_format
    elif parm[2] == 'Codigo':
        formato = cd_format
    elif parm[2] == 'Valor':
        formato = vl_format
    else:
        formato = de_format
    pagina.set_column(campo, campo, tamanho, formato)

# Formata Impressão
pagina.hide_gridlines(2)
pagina.freeze_panes(1, 0)
pagina.print_area(0,0,linhas, colunas)
pagina.repeat_rows(0,0)
pagina.fit_to_pages(1,0)
pagina.set_paper(9) # A4
pagina.set_landscape() # set_landscape or set_portrait()
pagina.center_horizontally()
#pagina.center_vertically()
pagina.set_margins(left=0.7, right=0.7, top=0.9, bottom=0.75)

# Formata Cabecalho e Rodape
cabecalho = '&L&[Picture]' \
          + '&C' + '&"Courier New,Bold"' \
                 + 'PREFEITURA DE MACEIÓ' \
                 + '\n\n' \
                 + 'SECRETARIA MUNICIPAL DE GESTAO' \
                 + '\n\n' \
                 + 'CONTRACHEQUE' \
          + '&R' + 'Data: &D'

logo_file = open('Prefeitura_Maceio_Logo.png', 'rb')
logo_data = BytesIO(logo_file.read())

pagina.set_header(cabecalho,
                  {'image_left': 'logo.png',
                   'image_data_left': logo_data})
#pagina.set_header(cabecalho, {'image_left': 'Prefeitura_Maceio_Logo.png'})
rodape = '&L' + 'SEMGE / AL – Sistema Informatizado de Gestão de Recursos Humanos' + ' ' + 'Módulo de PAGAMENTOS' \
       + '&R' +  'Página: &P of &N'
pagina.set_footer(rodape)

# Grava a Planilha
planilha.save()

Wall time: 512 ms
